In [3]:
import sys, os
os.environ['JAX_PLATFORMS'] = 'cpu'

import jax
from typing import Dict
import jax.numpy as jnp
import optax
import numpy as np
import math
from matplotlib import pyplot as plt
from jax_tqdm import scan_tqdm
from functools import partial
from jax.tree_util import tree_map
from dataclasses import dataclass
import eindex.array_api as EX
import einops

from jaxtorch import nn
from jaxtorch import PRNG, Context
import jaxtorch

from ngp.client import SpaceItem, OptimClient
from ngp.util import Partial

import optuna

In [ ]:
SWEEP_NAME='21-wooden-sweep'
study = optuna.create_study(
    study_name=SWEEP_NAME,
    load_if_exists=True,
    storage="sqlite:///../vqgan-jax/optuna.db",
    direction="minimize",
    sampler=optuna.samplers.GPSampler(),
)

pars = list(study.trials[0].params.keys())
dists = study.trials[0].distributions
trials = [trial for trial in study.trials if trial.state==1]
all_y = jnp.array([trial.value for trial in trials])
max_y = jnp.max(all_y[jnp.isfinite(all_y)])
# import json
# with open('../vqgan-jax/trials.json', 'r') as fp:
#     trials = json.load(fp)
#     all_y = jnp.array([trial[1] for trial in trials])
#     max_y = jnp.max(all_y[jnp.isfinite(all_y)])

# @dataclass
# class Trial:
#     params: Dict[str, float]
#     value: float

# trials = [Trial(params, value) for params, value in trials]
# pars = list(trials[0].params.keys())

/tmp/ipykernel_4088078/4166868394.py:7: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  sampler=optuna.samplers.GPSampler(),
[I 2025-02-25 04:11:41,022] Using an existing study with name '21-wooden-sweep' instead of creating a new one.


{'max_lr': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'wpe': FloatDistribution(high=10.0, log=True, low=0.1, step=None), 'wae': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.attn.q_proj': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.attn.k_proj': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.attn.v_proj': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.attn.c_proj': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.cross_attn.q_proj': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.cross_attn.o_proj': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.cross_attn.k_clip': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.cross_attn.v_clip': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.cross_attn.k_tags': FloatDistribution(high=10.0, log=True, low=0.1, step=None), '.cross_attn.v_tags': FloatDistribution(high=10.0, log=True, low=0.1, step=None),

In [ ]:
client = OptimClient('http://localhost:8000')
space = [
    SpaceItem(name, dist.low, dist.high, log=dist.log)
    for name, dist in (study.best_trial.distributions.items())
    if isinstance(dist, optuna.distributions.FloatDistribution)
]
sweep = client.new_sweep('vqgan-demo', space, objective='min')
print('id:', sweep.id)
for trial in trials:
    sweep.start(trial.params).report(trial.value if jnp.isfinite(trial.value) else max_y)


id: 606d0253-2a92-4ae4-9759-1f20c4241942


InvalidJSONError: Out of range float values are not JSON compliant: inf